In [1]:
import pandas as pd
import utils.hebrew_tokenizer as ht

df = pd.read_csv('./datasets/projectbenyehuda/benyehuda_nikud_dataset_filtered.csv')
# df = pd.read_csv('./datasets/projectbenyehuda/benyehuda_nikud_dataset_with_uncertainty.csv')

In [2]:
df.shape

(60315, 6)

In [3]:
import ast

def mask_str(mask):
    if isinstance(mask, str):
        mask = ast.literal_eval(mask)
    return "".join([str(x) for x in mask])

In [4]:
from tqdm.notebook import tqdm

mask_lengths = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Checking mask lengths"):
    mask_lengths.append(len(ht.get_nikud_mask(row['text'])[1]))

mismatch_rows = df[[mask_len != row['nikud_mask_length'] for mask_len, (_, row) in zip(mask_lengths, df.iterrows())]]
mismatch_rows.shape

Checking mask lengths:   0%|          | 0/60315 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2494 > 2048). Running this sequence through the model will result in indexing errors


(23, 6)

In [5]:
mismatch_rows['diff'] = [len(ht.get_nikud_mask(row['text'])[1]) - row['nikud_mask_length'] for _, row in mismatch_rows.iterrows()]
grouped = mismatch_rows.groupby('diff')
grouped.size()

C:\Users\eitam\AppData\Local\Temp\ipykernel_14548\2918813076.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mismatch_rows['diff'] = [len(ht.get_nikud_mask(row['text'])[1]) - row['nikud_mask_length'] for _, row in mismatch_rows.iterrows()]


diff
-1    23
dtype: int64

In [6]:
x = grouped.get_group(-1).iloc[10]
print(x['text'])
input_ids, nikud_mask = ht.get_nikud_mask(x['text'])
print(ht.tokenizer.decode(input_ids))
print(mask_str(nikud_mask))
print(mask_str(x['nikud_mask']))

&nbsp;↩ מולא צייטלין פרסם בעיתון “הבֹקר” כמו גם בעיתונים נוספים ציורים וקריקטורות
[CLS] & n b s p ; [UNK]   מ ו ל א   צ י י ט ל י ן   פ ר ס ם   ב ע י ת ו ן   “ ה ב ק ר ”   כ מ ו   ג ם   ב ע י ת ו נ י ם   נ ו ס פ י ם   צ י ו ר י ם   ו ק ר י ק ט ו ר ו ת [SEP]
0000000000000000000000000000000000001000000000000000000000000000000000000000000000
00000000000000000000000000000000000010000000000000000000000000000000000000000000000


In [7]:
sample = df.sample(1)
sample.iloc[0]

id                                                               24887
title                                אדוני המדבר תולדות הנבטים וממלכתם
text                 הוא מינה את הירונימוס, כותב ההיסטוריה, לפקח על...
nikud_mask           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
text_length                                                        131
nikud_mask_length                                                  132
Name: 16540, dtype: object

In [8]:
input_ids, nikud_mask = ht.get_nikud_mask(sample.iloc[0]['text'])
print(ht.tokenizer.decode(input_ids), len(mask_str(nikud_mask)), len(mask_str(sample.iloc[0]['nikud_mask'])))

[CLS] ה ו א   מ י נ ה   א ת   ה י ר ו נ י מ ו ס,   כ ו ת ב   ה ה י ס ט ו ר י ה,   ל פ ק ח   ע ל   נ י צ ו ל   ה א ס פ ל ט   ו צ י ו ו ה   ע ל י ו   ל ה כ י ן   ס י ר ו ת,   ל א ס ו ף   א ס פ ל ט   כ כ ל   ש י ו כ ל   ו ל ה ב י א   א ו ת ו   ל מ ק ו ם   מ ס ו י ם [SEP] 132 132


In [9]:

uncertainty_mask, ambiguous_chars = ht.get_nikud_uncertainty(sample.iloc[0]['text'])
sample.iloc[0]['text'], mask_str(sample.iloc[0]['nikud_mask']), mask_str(uncertainty_mask), ambiguous_chars

('הוא מינה את הירונימוס, כותב ההיסטוריה, לפקח על ניצול האספלט וציווה עליו להכין סירות, לאסוף אספלט ככל שיוכל ולהביא אותו למקום מסוּים',
 '000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000',
 '000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000011000000000000000000000000000000000000',
 [{'char': 'פ', 'position': (tensor(56), tensor(57))},
  {'char': 'פ', 'position': (tensor(93), tensor(94))},
  {'char': 'ל', 'position': (tensor(94), tensor(95))}])

In [17]:
small_df = df.sample(10000)

In [21]:
from tqdm.notebook import tqdm

tqdm.pandas(desc="Calculating uncertainty masks")
small_df['uncertainty_mask'] = small_df['text'].progress_apply(lambda text: ht.get_nikud_uncertainty(text)[0])


Calculating uncertainty masks:   0%|          | 0/10000 [00:00<?, ?it/s]

In [22]:
from tqdm.notebook import tqdm

tqdm.pandas()  # Enable tqdm for pandas

small_df['nikud_word_mask'] = small_df.progress_apply(
    lambda row: ht.convert_token_to_word_mask(row['text'], row['nikud_mask']), axis=1)
small_df['uncertainty_word_mask'] = small_df.progress_apply(
    lambda row: ht.convert_token_to_word_mask(row['text'], row['uncertainty_mask']), axis=1)

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [23]:
small_df.to_csv('./datasets/projectbenyehuda/benyehuda_nikud_dataset_with_uncertainty_word_mask.csv', index=False)

In [13]:
len(small_df.iloc[0]['uncertainty_word_mask']), len(small_df.iloc[0]['nikud_word_mask'])
# convert_token_to_word_mask(small_df.iloc[0]['text'], small_df.iloc[0]['uncertainty_mask'])
# convert_token_to_word_mask(small_df.iloc[0]['text'], small_df.iloc[0]['uncertainty_mask'])

(24, 24)

In [24]:

# Compute the overlap ratio in a separate step
def overlap_ratio(row):
    uncertainty_mask = row['uncertainty_word_mask']
    nikud_mask = row['nikud_word_mask']
    if isinstance(nikud_mask, str):
        nikud_mask = ast.literal_eval(nikud_mask)
    overlap = sum(1 for u, n in zip(uncertainty_mask, nikud_mask) if u == 1 and n == 1)
    total_nikud = sum(nikud_mask)
    return overlap / total_nikud if total_nikud > 0 else None

small_df['uncertainty_overlap_ratio'] = small_df.apply(overlap_ratio, axis=1)
small_df['uncertainty_overlap_ratio'].describe()

count    9688.000000
mean        0.433942
std         0.458009
min         0.000000
25%         0.000000
50%         0.250000
75%         1.000000
max         1.000000
Name: uncertainty_overlap_ratio, dtype: float64

In [25]:
not_enough_overlap = small_df[small_df['uncertainty_overlap_ratio'] <= 0.5]
test = not_enough_overlap.sample(5)

In [26]:
for i in range(len(test)):
    print(f"Example {i+1}:")
    print("Text:", test.iloc[i]['text'])
    print("Nikud Mask:      ", mask_str(test.iloc[i]['nikud_word_mask']))
    print("Uncertainty Mask:", mask_str(test.iloc[i]['uncertainty_word_mask']))
    print("Overlap Ratio:", test.iloc[i]['uncertainty_overlap_ratio'])
    print()

Example 1:
Text: רפי אוהב את הצבא (60–62) וחש שייכות, אך הסתייגותו מפיצוץ בתים ערביים (63) הופכת שאט נפש מהמִמסד, ממנהיגיו ומן התומכים בהם
Nikud Mask:       000000000000000010000
Uncertainty Mask: 000000000000001000000
Overlap Ratio: 0.0

Example 2:
Text: הוא הציע לה לעבור לביתו כ“אֶקוֹנוֹמית”, מנהלת משק-הבית
Nikud Mask:       00000100
Uncertainty Mask: 00000000
Overlap Ratio: 0.0

Example 3:
Text: [רוּחַ, הוֹ רוּחַ, מָה, בָּאת מֵרָחוֹק?] / כריסטינה רוזטי / עופרה עופר אורן[רוּחַ, הוֹ רוּחַ, מָה, בָּאת מֵרָחוֹק?] מאת כריסטינה רוזטי תורגם מאנגלית מאת עופרה עופר אורן רוּחַ, הוֹ רוּחַ, מָה, בָּאת מֵרָחוֹק
Nikud Mask:       101111000000101111000000000101111
Uncertainty Mask: 000000001010000000001000000000000
Overlap Ratio: 0.0

Example 4:
Text: שבת אחת לפנות־ערב חזרנו, רוב ילדי הכיתה, ממסיבת יום־הולדת אשר פוליק, מִסיבה כלשהי, לא השתתף בה
Nikud Mask:       0000000000010000
Uncertainty Mask: 0000000000000000
Overlap Ratio: 0.0

Example 5:
Text: אין זה ‘בלש־חידה’ (puzzle), שכּן מראש ידוע מי אש

In [80]:
_, ambiguous = ht.get_nikud_uncertainty(not_enough_overlap.iloc[0]['text'])
ambiguous

[{'char': 'ק',
  'position': (tensor(5), tensor(6)),
  'entropy': 1.7002260453349685,
  'margin': 0.10639014840126038,
  'max_prob': 0.46183285117149353,
  'top_candidates': [('ְּ', 0.46183285117149353),
   ('ָּ', 0.35544270277023315),
   ('ַּ', 0.14430095255374908),
   ('ִּ', 0.025373805314302444),
   ('', 0.004338645841926336)]},
 {'char': 'מ',
  'position': (tensor(6), tensor(7)),
  'entropy': 1.8923375398731663,
  'margin': 0.3622446358203888,
  'max_prob': 0.5610603094100952,
  'top_candidates': [('ָ', 0.5610603094100952),
   ('ְ', 0.19881567358970642),
   ('ָּ', 0.1026516854763031),
   ('ֵ', 0.0913892388343811),
   ('ַ', 0.023114318028092384)]},
 {'char': 'ל',
  'position': (tensor(28), tensor(29)),
  'entropy': 1.007569559218598,
  'margin': 0.2989606559276581,
  'max_prob': 0.6457830667495728,
  'top_candidates': [('ַ', 0.6457830667495728),
   ('ָ', 0.3468224108219147),
   ('ְ', 0.0030318722128868103),
   ('', 0.0022456843871623278),
   ('ַּ', 0.0006822854629717767)]},
 {'char'